In [2]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler

def transforming(columns, df):
    for column in columns:
        encoder = LabelEncoder()
        df[column] = encoder.fit_transform(df[column])


df = pd.read_csv("bird_migration_data.csv").drop(columns=["Bird_ID","Tagged_By", "Tag_Type", "Migration_Interrupted", "Interrupted_Reason", "Migration_Success", "Observation_Quality", "Tracking_Quality", "Recovery_Location_Known"], axis=1)


transforming(columns=["Species", "Region", "Habitat", "Weather_Condition", "Migration_Reason", "Nesting_Success","Migration_Start_Month", "Migration_End_Month", "Migrated_in_Flock", "Food_Supply_Level"], df=df)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

model = keras.Sequential([
    keras.layers.Input(shape=(31,)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.3), 
    keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.3), 
    keras.layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1, activation="linear")
])

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping # type: ignore


X = df.drop(["Flight_Distance_km"], axis=1)

scaler = StandardScaler()
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

y = df["Flight_Distance_km"]
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer = optimizer, loss = "mse", metrics=["mae"])
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=20, callbacks=[early_stopping])

0       1753.79
1       3263.05
2       2871.65
3       2317.38
4       3077.38
         ...   
9995    2357.69
9996    2901.75
9997    3019.80
9998    2663.78
9999    2549.41
Name: Flight_Distance_km, Length: 10000, dtype: float64
Epoch 1/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 597us/step - loss: 2425351.0000 - mae: 1212.6604 - val_loss: 301180.5312 - val_mae: 438.5372
Epoch 2/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss: 377399.5938 - mae: 488.8181 - val_loss: 301104.3125 - val_mae: 438.3899
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - loss: 364199.1250 - mae: 481.7448 - val_loss: 285879.8750 - val_mae: 426.1597
Epoch 4/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 355508.5625 - mae: 477.8855 - val_loss: 292124.5938 - val_mae: 430.5333
Epoch 5/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - loss: 344429.1250 - mae: 467.6586 - val_loss: 284558.6250 - val_mae: 424.0364
Epoch 6/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 358256.9375 - mae: 480.1606 